In [2]:
import numpy as np
import logging 
import sys, time
import os
import h5py
sys.path.append('/Users/lekan/catkin_ws/src/dp_planning/scripts')

logger = logging.getLogger(__name__)

In [2]:
with open('../scripts/moplan_data.csv', 'r+') as f:
    data = f.readlines()
moplan_data = [x.split() for x in data]    

In [3]:
proper_data = []
for i in range(len(moplan_data)):
    if not moplan_data[i]:
        continue
    temp = moplan_data[i]
    to_append = []
    temp = [x.split(sep=',')[0] for x in temp]
    #print(temp)
    for x in temp:
        if '[' in x:
            x = x.split(sep="[")[1]
        elif ']' in x:
            x = x.split(sep=']')[0]
        to_append.append(float(x))
    proper_data.append(to_append)

proper_data = np.array(proper_data)
print(proper_data.shape)

(21, 14)


In [4]:
joint_workspace = dict(pts1=dict(q=[], qdot=[]), pts2=dict(q=[], qdot=[]), pts3=dict(q=[], qdot=[]))

In [5]:
joint_workspace['pts1']['q'] = proper_data[slice(0, 21, 3), slice(0, 7)]
joint_workspace['pts1']['qdot'] = proper_data[slice(0, 21, 3), slice(7, 14)]
joint_workspace['pts2']['q'] = proper_data[slice(1, 21, 3), slice(0, 7)]
joint_workspace['pts2']['qdot'] = proper_data[slice(1, 21, 3), slice(7, 14)]
joint_workspace['pts3']['q'] = proper_data[slice(2, 21, 3), slice(0, 7)]
joint_workspace['pts3']['qdot'] = proper_data[slice(2, 21, 3), slice(7, 14)]

In [15]:
qs = np.vstack([joint_workspace['pts1']['q'], joint_workspace['pts2']['q']])
qdots = np.vstack([joint_workspace['pts1']['qdot'], joint_workspace['pts2']['qdot']])

In [16]:
# print(joint_workspace['pts1']['q'])
# print()
# print(joint_workspace['pts2']['q'])
# print()
# print(joint_workspace['pts3']['q'])

np.set_printoptions(precision=4)
# qs = np.hstack([joint_workspace['pts1']['q'], joint_workspace['pts2']['q']])
print(qs.shape)

(14, 7)


### sanity check

In [52]:
### save the data we have collected
filename = '../scripts/{}.h5'.format('joints_data')
os.remove(filename) if os.path.isfile(filename) else None
time.sleep(4)
with h5py.File(filename, 'w') as f:
    pos_grp = f.create_group('workspace_coords')
    pos_grp.create_dataset("joint_positions", data=qs, dtype=np.float32, compression="gzip", compression_opts=9)
    pos_grp.create_dataset("joint_velocities", data=qdots,  dtype=np.float32, compression="gzip", compression_opts=9)
    targ_grp = f.create_group('workspace_targets')
    targ_grp.create_dataset("joint_positions", data=joint_workspace['pts3']['q'],  dtype=np.float32, compression="gzip", compression_opts=9)
    targ_grp.create_dataset("joint_velocities", data=joint_workspace['pts3']['qdot'],  dtype=np.float32, compression="gzip", compression_opts=9)
    

In [4]:
a = np.random.randn(4, 7)
for i in range(7):
    a[:, i] = i
    
print(a)

[[0. 1. 2. 3. 4. 5. 6.]
 [0. 1. 2. 3. 4. 5. 6.]
 [0. 1. 2. 3. 4. 5. 6.]
 [0. 1. 2. 3. 4. 5. 6.]]


In [9]:
for i in range(7):
#     for (x, y, xd, yd) in (a[:,i]):
    x, y, xd, yd = a[:,i]
    print(x, y, xd, yd)

0.0 0.0 0.0 0.0
1.0 1.0 1.0 1.0
2.0 2.0 2.0 2.0
3.0 3.0 3.0 3.0
4.0 4.0 4.0 4.0
5.0 5.0 5.0 5.0
6.0 6.0 6.0 6.0
